In [1]:
from pathlib import Path
import pandas as pd

In [2]:
#create empty dataframe
column_names = {"user_id":'',"stars":int(),"text":''}
review_df = pd.DataFrame(column_names, index = [])
review_df["stars"] = review_df["stars"].astype("int8")
review_df.dtypes

user_id    object
stars        int8
text       object
dtype: object

In [3]:
review_path = Path(r"D:\yelp_data\yelp_academic_dataset_review.json")
review_holder = pd.read_json(review_path,lines = True, orient ="records", nrows = 1000000, chunksize=100000)
for i in review_holder:
    i = i[["user_id","stars","text"]]
    review_df = review_df.append(i, ignore_index = True)
    review_df["stars"] = review_df["stars"].astype("int8")
print(review_df.shape)

(1000000, 3)


In [4]:
review_df.head()

,user_id,stars,text
0,ak0TdVmGKo4pwqdJSTLwWw,4,Apparently Prides Osteria had a rough summer a...
1,YoVfDbnISlW0f7abNQACIg,4,This store is pretty good. Not as great as Wal...
2,eC5evKn1TWDyHCyQAwguUw,5,I called WVM on the recommendation of a couple...
3,SFQ1jcnGguO0LYWnbbftAA,2,I've stayed at many Marriott and Renaissance M...
4,0kA0PAJ8QFMeveQWHFqz2A,4,The food is always great here. The service fro...


In [5]:
review_df.dtypes

user_id    object
stars        int8
text       object
dtype: object

In [6]:
review_df.memory_usage()

Index          128
user_id    8000000
stars      1000000
text       8000000
dtype: int64

In [7]:
review_df.memory_usage().sum()

17000128

In [13]:
#review number of unique users, find those who have 5 or more reviews
user_summary = review_df.groupby("user_id").agg({'stars':['sum','count']}).droplevel(level=0, axis =1)
user_summary= user_summary.loc[user_summary['count'] > 4]
user_summary["average_score"] = user_summary["sum"]/user_summary["count"]
print(user_summary.shape)
user_summary.head()

(29115, 3)


,sum,count,average_score
user_id,,,
--3l8wysfp49Z2TLnyT0vg,19.0,6,3.166667
--Cq0V6vgQQDU8oQEa0NGQ,25.0,6,4.166667
--u09WAjW741FdfkJXxNmg,49.0,12,4.083333
-012EkqHKaqTS3ARkcgavw,16.0,5,3.200000
-0AyZxS5C--WySnbW_Q8yQ,23.0,5,4.600000


In [14]:
#modify main DF to get those with more than 5 reviews
review_df = review_df.join(user_summary["average_score"], how = "inner", on = "user_id")
review_df.shape

(293754, 4)

In [17]:
#look at low average score reviewers
low_reviewers = user_summary.loc[user_summary["average_score"] < 2.5]
print(low_reviewers.shape)
low_reviewers.head()

(1179, 3)


,sum,count,average_score
user_id,,,
-4HoJxvbaytZ_wXES6-sJg,11.0,5,2.200000
-DQG0iQsVjjvDbbc1-VvMg,10.0,5,2.000000
-Dz9WQ6UpS8VyXOhn2z86Q,14.0,6,2.333333
-GTDgCPa0LkoDBATv4Qptw,9.0,5,1.800000
-NfBbOl6rKw2zVMewfZY5A,12.0,5,2.400000
